In [7]:
import os
os.chdir("..")
print("Current Directory:", os.getcwd())

Current Directory: /workspace/iscat


In [2]:
def get_fluo_paths(root_path:str, mode:str="Brightfield"):
    """
    Extract paths to .nd2 files and corresponding TIFF files from the specified mode folder.

    Args:
        root_path (str): The root directory to search.
        mode (str): The folder name to focus on (default is 'Brightfield').

    Returns:
        tuple: Two lists - list of .nd2 file paths and list of tuples with corresponding TIFF file paths.
    """

    target_files = []

    for dirpath, dirnames, filenames in os.walk(root_path):
        if os.path.basename(dirpath) == mode:

            # Generate TIFF file paths dynamically based on the prefix
            cy5_path = os.path.join(dirpath, f'Captured Cy5.tif')
            fitc_path = os.path.join(dirpath, f'Captured FITC.tif')
            tritc_path = os.path.join(dirpath, f'Captured TRITC.tif')
            target_files.append((cy5_path, fitc_path, tritc_path))
            
            # Ensure all three TIFF files exist
            assert all(os.path.exists(path) for path in [cy5_path, fitc_path, tritc_path])
        
            
    return target_files

In [3]:
base_path = os.path.join("data", "2024_11_29","Metasurface","Chip_02")
target_files= get_fluo_paths(base_path, 'Brightfield')

In [4]:
target_files

[('data/2024_11_29/Metasurface/Chip_02/Metasurface 01/Brightfield/Captured Cy5.tif',
  'data/2024_11_29/Metasurface/Chip_02/Metasurface 01/Brightfield/Captured FITC.tif',
  'data/2024_11_29/Metasurface/Chip_02/Metasurface 01/Brightfield/Captured TRITC.tif'),
 ('data/2024_11_29/Metasurface/Chip_02/Metasurface 09/Brightfield/Captured Cy5.tif',
  'data/2024_11_29/Metasurface/Chip_02/Metasurface 09/Brightfield/Captured FITC.tif',
  'data/2024_11_29/Metasurface/Chip_02/Metasurface 09/Brightfield/Captured TRITC.tif'),
 ('data/2024_11_29/Metasurface/Chip_02/Metasurface 05/Brightfield/Captured Cy5.tif',
  'data/2024_11_29/Metasurface/Chip_02/Metasurface 05/Brightfield/Captured FITC.tif',
  'data/2024_11_29/Metasurface/Chip_02/Metasurface 05/Brightfield/Captured TRITC.tif'),
 ('data/2024_11_29/Metasurface/Chip_02/Metasurface 07/Brightfield/Captured Cy5.tif',
  'data/2024_11_29/Metasurface/Chip_02/Metasurface 07/Brightfield/Captured FITC.tif',
  'data/2024_11_29/Metasurface/Chip_02/Metasurface 0

In [5]:
from src.data_processing.utils import Utils
Utils.generate_np_masks(target_files,seg_args=None,seg_method="comdet")

Creating Masks with ComDet: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [06:42<00:00, 44.76s/it]


In [17]:
a={
    "a":1,
    "c":3,
    "b":2,
    

}
print(a.items())

dict_items([('a', 1), ('c', 3), ('b', 2)])


In [3]:
import h5py
import numpy as np
import torch
from typing import Tuple, Dict, List

def convert_instance_masks_to_maskrcnn_format(
    instance_masks: np.ndarray
) -> Tuple[List[torch.Tensor], List[torch.Tensor], List[torch.Tensor]]:
    """
    Convert instance segmentation masks to MaskRCNN format.
    
    Args:
        instance_masks: Array of shape (C, H, W) where C is number of classes,
                       each channel contains instance masks with unique IDs
    
    Returns:
        Tuple containing:
        - masks: List of binary masks for each instance
        - boxes: List of bounding boxes in [x1, y1, x2, y2] format
        - labels: List of class labels for each instance
    """
    height, width = instance_masks.shape[1:]
    masks = []
    boxes = []
    labels = []
    
    # Process each class
    for class_idx in range(instance_masks.shape[0]):
        class_mask = instance_masks[class_idx]
        unique_instances = np.unique(class_mask)
        
        # Skip background (0)
        for instance_id in unique_instances[unique_instances > 0]:
            # Create binary mask for this instance
            binary_mask = (class_mask == instance_id)
            masks.append(torch.from_numpy(binary_mask))
            
            # Get bounding box
            y_indices, x_indices = np.where(binary_mask)
            if len(y_indices) > 0 and len(x_indices) > 0:
                x1, x2 = x_indices.min(), x_indices.max()
                y1, y2 = y_indices.min(), y_indices.max()
                boxes.append(torch.tensor([x1, y1, x2, y2], dtype=torch.float32))
                
                # Add class label (add 1 because MaskRCNN uses 0 as background)
                labels.append(torch.tensor(class_idx + 1))
    
    # Handle empty cases
    if not masks:
        return [], torch.zeros((0, 4)), torch.zeros(0)
    
    return masks, torch.stack(boxes), torch.stack(labels)

def process_image_for_maskrcnn(
    image: np.ndarray,
    normalize: bool = True
) -> torch.Tensor:
    """
    Process the image for MaskRCNN input.
    
    Args:
        image: numpy array of shape (Z, H, W)
        normalize: whether to normalize the image
    
    Returns:
        torch.Tensor: Processed image tensor
    """
    # Convert to torch tensor
    image = torch.from_numpy(image).float()
    
    # Handle Z-stack: either take max projection or specific slice
    if image.ndim == 3:
        image = image.max(dim=0)[0]  # max projection
        # Alternatively: image = image[middle_slice]
    
    # Add channel dimension
    image = image.unsqueeze(0)
    
    # Normalize if requested
    if normalize:
        image = (image - image.mean()) / (image.std() + 1e-7)
    
    return image

def load_sample_from_hdf5(
    hdf5_path: str,
    idx: int,
    normalize_image: bool = True
) -> Tuple[torch.Tensor, Dict]:
    """
    Load a single sample from HDF5 file in MaskRCNN format.
    
    Args:
        hdf5_path: Path to HDF5 file
        idx: Index of the sample to load
        normalize_image: Whether to normalize the image
    
    Returns:
        Tuple containing:
        - image: Tensor of shape (C, H, W)
        - target: Dictionary containing:
            - boxes: Tensor of shape (N, 4) - coordinates of N bounding boxes
            - labels: Tensor of shape (N,) - class labels for each box
            - masks: List of N binary masks
            - image_id: Tensor containing image index
            - area: Tensor of shape (N,) - area of each mask
            - iscrowd: Tensor of shape (N,) - set to 0 for all instances
    """
    with h5py.File(hdf5_path, 'r') as f:
        # Load image and instance masks
        image = f['image_patches'][idx]
        instance_masks = f['instance_masks'][idx]
    
    # Process image
    processed_image = process_image_for_maskrcnn(image, normalize_image)
    
    # Convert instance masks to MaskRCNN format
    masks, boxes, labels = convert_instance_masks_to_maskrcnn_format(instance_masks)
    
    # Calculate areas for each instance
    if masks:
        areas = torch.tensor([mask.sum().float() for mask in masks])
        iscrowd = torch.zeros((len(masks),), dtype=torch.int64)
    else:
        areas = torch.zeros((0,), dtype=torch.float32)
        iscrowd = torch.zeros((0,), dtype=torch.int64)
    
    # Create target dictionary
    target = {
        'boxes': boxes,
        'labels': labels,
        'masks': masks,
        'image_id': torch.tensor([idx]),
        'area': areas,
        'iscrowd': iscrowd
    }
    
    return processed_image, target


In [9]:
processed_image, target= load_sample_from_hdf5('dataset/brightfield_instance.hdf5',0)

In [27]:
with h5py.File(path, 'r') as f:
    # Load image and instance masks
    image = f['image_patches'][0]
    instance_masks = f['instance_masks'][0]

In [31]:
instance_masks

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8)

In [25]:
path = 'dataset/brightfield_instance.hdf5'
def load_sample_from_hdf5(file_path, index):
    with h5py.File(file_path, 'r') as f:
        image = f['images'][index]  # Assuming 'images' is the dataset key
        boxes = f['boxes'][index]   # (N, 4) format: [x_min, y_min, x_max, y_max]
        labels = f['labels'][index] # Class labels for each instance
        masks = f['masks'][index]   # Binary masks (N, H, W)
        
    return image, {'boxes': boxes, 'labels': labels, 'masks': masks}
def plot_instance_segmentation(image, target):
    # Normalize the 16-bit image for visualization
    image = image.astype(np.float32) / (2**16)
    
    # Convert grayscale to RGB for visualization
    image_rgb = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_GRAY2RGB)
    
    # Get segmentation data
    boxes = target['boxes']
    labels = target['labels']
    masks = target['masks']
    
    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)
    
    # Random color generator
    def random_color():
        return tuple(random.choices(range(256), k=3))
    
    for i in range(len(boxes)):
        x_min, y_min, x_max, y_max = boxes[i]
        color = random_color()
        
        # Draw bounding box
        cv2.rectangle(image_rgb, (int(x_min), int(y_min)), (int(x_max), int(y_max)), color, 2)
        plt.text(x_min, y_min - 5, f'Class {labels[i]}', color='yellow', fontsize=8, bbox=dict(facecolor='black', alpha=0.5))
        
        # Overlay the segmentation mask with transparency
        mask = masks[i].astype(np.uint8)
        colored_mask = np.zeros_like(image_rgb)
        colored_mask[:, :, 0] = mask * color[0]  # Red channel
        colored_mask[:, :, 1] = mask * color[1]  # Green channel
        colored_mask[:, :, 2] = mask * color[2]  # Blue channel
        
        image_rgb = cv2.addWeighted(image_rgb, 1, colored_mask, 0.5, 0)
    
    plt.imshow(image_rgb)
    plt.axis("off")
    plt.title("Instance Segmentation Visualization")
    plt.show()


In [26]:
processed_image, target = load_sample_from_hdf5('dataset/brightfield_instance.hdf5', 0)
plot_instance_segmentation(processed_image, target)

KeyError: "Unable to synchronously open object (object 'images' doesn't exist)"